## Random Forest

In [75]:
from sklearn.metrics import f1_score,make_scorer
import pandas as pd
import numpy as np
import sklearn.ensemble as kn
from sklearn.model_selection import GridSearchCV
import collections as cls

In [42]:
#Import train and test set:
X_train = pd.read_csv('X_train_8.csv')
X_test = pd.read_csv('X_test_2.csv')
y_train = pd.read_csv('y_train.csv',header=None)
y_test = pd.read_csv('y_test.csv',header=None)


In [48]:
rf=kn.RandomForestClassifier()
tree_num=[50,100,150,200,250,300,350,400,450,500,550,600]
max_depth=[7,8,9,10,11,12,13]
parameters ={'n_estimators':tree_num,'max_depth':max_depth}
f1 = make_scorer(f1_score, average='micro')
clf = GridSearchCV(rf,parameters,cv=5,scoring=f1)
clf.fit(X_train,np.array(y_train).ravel())

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600], 'max_depth': [7, 8, 9, 10, 11, 12, 13]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score, average=micro), verbose=0)

For random forest, we should tune two parameters: number of trees and max deepth for each tree.  We use five folders cross validation in trainning set to choose the best parameters combination.

In [50]:
clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=350, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [51]:
clf.best_params_

{'max_depth': 13, 'n_estimators': 350}

In [52]:
clf.best_score_

0.67957054799160066

From the result, we can find the best parameters are 

In [71]:
best_model=kn.RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=350, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [72]:
best_model.fit(X_train,np.array(y_train))

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=350, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [73]:
y_hat=best_model.predict(X_test)

In [74]:
acc=np.mean([y_hat[i]==y_test.iloc[i,0] for i in range(len(y_hat))])
print acc

0.686107159791


In [76]:
cls.Counter(y_hat)

Counter({7: 12156, 10: 2607})

In [77]:
acc

0.68610715979137027

In [78]:
pd.DataFrame(y_hat).to_csv('y_rf.csv')
y_pred=pd.read_csv('y_rf.csv')

In [79]:
def pred_ensembel(preds,weights):
    '''This function will get the final predict by ensembling
    Input:
        preds: prediction of models
        weights:weight for each model
    '''
    weighted_pred=[]
    n=preds.shape[0]
    p=preds.shape[1]
    for i in range(n):
        temp=[]
        for j in range(p):
            temp+=[preds.iloc[i,j]]*weights[j]
            counts=cls.Counter(np.array(temp))
        label_temp=list(counts.most_common(1))
        label=label_temp[0][0]
        weighted_pred.append(label)
    return weighted_pred
            

In [80]:
final_pred=pred_ensembel(y_pred,[1,2,1])

In [81]:
np.mean(final_pred==y_test)

0    0.686107
dtype: float64

In [82]:
cls.Counter(final_pred)

Counter({7: 12156, 10: 2607})